In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import shutil
from keras import layers
from keras import losses
from keras.regularizers import L2
from keras import metrics
from sklearn.metrics import confusion_matrix
import tensorflow_probability as tfp
from keras import optimizers
os.chdir("/content/drive/MyDrive/Colab Notebooks/PCOSGen-train")

In [ ]:
dataset_path = r"/content/drive/MyDrive/Colab Notebooks/PCOSGen-train/class_label.xlsx"
df = pd.read_excel(dataset_path)
df.head()

,imagePath,Healthy
0,100image13.jpg,1
1,100image2.jpg,1
2,100image65.jpg,0
3,100image71.jpg,0
4,100image83.jpg,0


In [ ]:
directory_path = r"/content/drive/My Drive/Colab Notebooks/PCOSGen-train/Image Dataset"
CLASS_NAMES = ["Healthy", "Unhealthy"]
training_dataset, valid_dataset = tf.keras.utils.image_dataset_from_directory(
    directory_path,
    labels = 'inferred',
    label_mode = 'binary',
    color_mode = 'rgb',
    batch_size = 32,
    image_size = (256,256),
    class_names = CLASS_NAMES,
    shuffle = True,
    seed = 42,
    validation_split = 0.2,
    subset = 'both'
)
# plt.figure(figsize=(10, 10))
# for images, labels in training_dataset.take(1):
#   for i in range(16):
#     ax = plt.subplot(4, 4, i+1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(CLASS_NAMES[labels[i].numpy().astype('uint8')[0]])
#     plt.axis("off")
# plt.show()
resize_rescale_layers = tf.keras.Sequential([
    layers.Resizing(256,256),
    layers.Rescaling(1./255)
])
augment_layers = tf.keras.Sequential([
    layers.RandomRotation(factor = (-0.025,0.025),),
    layers.RandomFlip(mode = 'horizontal',),
    layers.RandomContrast(factor = 0.1),
])

def augment_layer(image, label):
  return augment_layers(image, training = True), label
training_dataset = (training_dataset.map(augment_layer, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE))
valid_dataset = (valid_dataset.prefetch(tf.data.AUTOTUNE))
CONFIGURATION = {
    "BATCH_SIZE" : 32,
    "IM_SIZE" : 256,
    "LEARNING_RATE" : 0.001,
    "N_EPOCHS" : 100,
    "DROPOUT_RATE" : 0.0,
    "REGULARIZATION_RATE" : 0.0,
    "N_FILTERS" : 6,
    "KERNEL_SIZE" : 3,
    "N_STRIDES" : 1,
    "POOL_SIZE" : 2,
    "N_DENSE_1" : 1024,
    "N_DENSE_2" : 128,
    "NUM_CLASSES" : 2
}
lenet_model = tf.keras.Sequential([
    layers.InputLayer(input_shape = (None,None,3)),
    resize_rescale_layers,

    layers.Conv2D(filters = CONFIGURATION['N_FILTERS'], kernel_size = CONFIGURATION['KERNEL_SIZE'], strides = CONFIGURATION['N_STRIDES'], activation = 'relu', kernel_regularizer=L2(CONFIGURATION['REGULARIZATION_RATE'])),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size = CONFIGURATION['POOL_SIZE'], strides = CONFIGURATION['N_STRIDES']*2),
    layers.Dropout(rate = CONFIGURATION['DROPOUT_RATE']),

    layers.Conv2D(filters = CONFIGURATION['N_FILTERS']*2 + 4, kernel_size = CONFIGURATION['KERNEL_SIZE'], strides = CONFIGURATION['N_STRIDES'], activation = 'relu', kernel_regularizer=L2(CONFIGURATION['REGULARIZATION_RATE'])),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size = CONFIGURATION['POOL_SIZE'], strides = CONFIGURATION['N_STRIDES']*2),

    layers.Flatten(),
    layers.Dense(CONFIGURATION['N_DENSE_1'], activation='relu', kernel_regularizer=L2(CONFIGURATION['REGULARIZATION_RATE'])),
    layers.BatchNormalization(),
    layers.Dropout(rate=CONFIGURATION['DROPOUT_RATE']),

    layers.Dense(CONFIGURATION['N_DENSE_2'],activation = 'relu', kernel_regularizer=L2(CONFIGURATION['REGULARIZATION_RATE'])),
    layers.BatchNormalization(),

    layers.Dense(1, activation = 'sigmoid')

])
lenet_model.summary()
lenet_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Found 3200 files belonging to 2 classes.
Using 2560 files for training.
Using 640 files for validation.
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 6)       168       
                                                                 
 batch_normalization (Batch  (None, 254, 254, 6)       24        
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 6)       0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 127, 127, 6)       0         
                

In [ ]:
backbone = tf.keras.applications.efficientnet.EfficientNetB4(
    include_top = False,
    weights = 'imagenet',
    input_shape = (256, 256, 3),
)

inception = tf.keras.applications.inception_v3.InceptionV3(
    include_top = False,
    weights = 'imagenet',
    input_shape = (256, 256, 3),
)

backbone.trainable = False

inception.trainable = False

def inceptionModule_1(x):
  conv_1x1 = layers.Conv2D(6, kernel_size = (1, 1), strides = (1, 1), activation = 'relu')(x)
  conv_2x2 = layers.Conv2D(6, kernel_size = (3, 3), strides = (1, 1), activation = 'relu')(x)
  conv_5x5 = layers.Conv2D(6, kernel_size = (5, 5), strides = (1, 1), activation = 'relu')(x)

  conv_1x1 = layers.Resizing(256,256)(conv_1x1)
  conv_2x2 = layers.Resizing(256,256)(conv_2x2)
  conv_5x5 = layers.Resizing(256,256)(conv_5x5)

  inceptionModule_1 = layers.concatenate([conv_1x1, conv_2x2, conv_5x5])

  return inceptionModule_1

def inceptionModule_2(x):
  conv_1x1  = layers.Conv2D(6, kernel_size = (1, 1), strides = (1, 1), activation = 'relu')(x)

  conv_1x1_2 = layers.Conv2D(6, kernel_size = (1, 1), strides = (1, 1), activation = 'relu')(x)
  conv_1x1_2 = layers.Conv2D(6, kernel_size = (1, 1), strides = (1, 1), activation = 'relu')(conv_1x1_2)

  conv_3x3 = layers.Conv2D(6, kernel_size = (3, 3), strides = (1, 1), activation = 'relu')(x)
  conv_3x3 = layers.Conv2D(6, kernel_size = (1, 1), strides = (1, 1), activation = 'relu')(conv_3x3)

  conv_5x5 = layers.Conv2D(6, kernel_size = (5, 5), strides = (1, 1), activation = 'relu')(x)
  conv_5x5 = layers.Conv2D(6, kernel_size = (1, 1), strides = (1, 1), activation = 'relu')(conv_5x5)

  conv_1x1 = layers.Resizing(256,256)(conv_1x1)
  conv_1x1_2 = layers.Resizing(256,256)(conv_1x1_2)
  conv_3x3 = layers.Resizing(256,256)(conv_3x3)
  conv_5x5 = layers.Resizing(256,256)(conv_5x5)


  inceptionModule_2 = layers.concatenate([conv_1x1, conv_1x1_2, conv_3x3, conv_5x5])

  return inceptionModule_2

inputs = layers.Input(shape = (256,256,3)),
x = backbone(inputs)

x = inceptionModule_1(x)
x = inceptionModule_2(x)

x = layers.GlobalAveragePooling2D()(x)
output = layers.Dense(1, activation = 'sigmoid')(x)

model_2 = tf.keras.Model(inputs = inputs, outputs = output, name = 'test')

model_2.summary()

Model: "test"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 efficientnetb4 (Functional  (None, 8, 8, 1792)           1767382   ['input_4[0][0]']             
 )                                                        3                                       
                                                                                                  
 conv2d_96 (Conv2D)          (None, 8, 8, 6)              10758     ['efficientnetb4[0][0]']      
                                                                                                  
 conv2d_97 (Conv2D)          (None, 6, 6, 6)              96774     ['efficientnetb4[0][0]']   

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

X = np.concatenate([image.numpy() for image, _ in training_dataset])
y = np.concatenate([label.numpy() for _, label in training_dataset])

num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

backbone = tf.keras.applications.efficientnet.EfficientNetB4(
    include_top=False,
    weights='imagenet',
    input_shape=(256, 256, 3),
)
backbone.trainable = False

inception = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=(256, 256, 3),
)
inception.trainable = False

def inceptionModule_1(x):
  conv_1x1 = layers.Conv2D(6, kernel_size = (1, 1), strides = (1, 1), activation = 'relu')(x)
  conv_2x2 = layers.Conv2D(6, kernel_size = (3, 3), strides = (1, 1), activation = 'relu')(x)
  conv_5x5 = layers.Conv2D(6, kernel_size = (5, 5), strides = (1, 1), activation = 'relu')(x)

  conv_1x1 = layers.Resizing(256,256)(conv_1x1)
  conv_2x2 = layers.Resizing(256,256)(conv_2x2)
  conv_5x5 = layers.Resizing(256,256)(conv_5x5)

  inceptionModule_1 = layers.concatenate([conv_1x1, conv_2x2, conv_5x5])

  return inceptionModule_1

def inceptionModule_2(x):
  conv_1x1  = layers.Conv2D(6, kernel_size = (1, 1), strides = (1, 1), activation = 'relu')(x)

  conv_1x1_2 = layers.Conv2D(6, kernel_size = (1, 1), strides = (1, 1), activation = 'relu')(x)
  conv_1x1_2 = layers.Conv2D(6, kernel_size = (1, 1), strides = (1, 1), activation = 'relu')(conv_1x1_2)

  conv_3x3 = layers.Conv2D(6, kernel_size = (3, 3), strides = (1, 1), activation = 'relu')(x)
  conv_3x3 = layers.Conv2D(6, kernel_size = (1, 1), strides = (1, 1), activation = 'relu')(conv_3x3)

  conv_5x5 = layers.Conv2D(6, kernel_size = (5, 5), strides = (1, 1), activation = 'relu')(x)
  conv_5x5 = layers.Conv2D(6, kernel_size = (1, 1), strides = (1, 1), activation = 'relu')(conv_5x5)

  conv_1x1 = layers.Resizing(256,256)(conv_1x1)
  conv_1x1_2 = layers.Resizing(256,256)(conv_1x1_2)
  conv_3x3 = layers.Resizing(256,256)(conv_3x3)
  conv_5x5 = layers.Resizing(256,256)(conv_5x5)

inputs = tf.keras.layers.Input(shape=(256, 256, 1))
backbone_out = backbone(inputs)
inceptionModule1_out = inceptionModule_1(backbone_out)
inceptionModule2_out = inceptionModule_2(inceptionModule1_out)
global_avg_pooling = tf.keras.layers.GlobalAveragePooling2D()(inceptionModule2_out)
output = tf.keras.layers.Dense(1, activation='sigmoid')(global_avg_pooling)

model = tf.keras.Model(inputs=inputs, outputs=output)

for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(f"Fold {fold + 1}:")

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

    results = model.evaluate(X_test, y_test)
    print(f"Fold {fold + 1} - Test Loss: {results[0]}, Test Accuracy: {results[1]}")

TypeError: Inputs to a layer should be tensors. Got 'None' (of type <class 'NoneType'>) as input for layer 'global_average_pooling2d_1'.

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

X = np.concatenate([image.numpy() for image, _ in training_dataset])
y = np.concatenate([label.numpy() for _, label in training_dataset])

num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)


In [ ]:
for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(f"Fold {fold + 1}:")

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model_2.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

    results = model_2.evaluate(X_test, y_test)
    print(f"Fold {fold + 1} - Test Loss: {results[0]}, Test Accuracy: {results[1]}")

Fold 1:
Epoch 1/10
64/64 [==============================] - 57s 523ms/step - loss: 0.6129 - accuracy: 0.7046 - val_loss: 0.6331 - val_accuracy: 0.7090
Epoch 2/10
64/64 [==============================] - 26s 405ms/step - loss: 0.5997 - accuracy: 0.7144 - val_loss: 0.6153 - val_accuracy: 0.7148
Epoch 3/10
64/64 [==============================] - 26s 400ms/step - loss: 0.5905 - accuracy: 0.7144 - val_loss: 0.6214 - val_accuracy: 0.7148
Epoch 4/10
64/64 [==============================] - 26s 401ms/step - loss: 0.5892 - accuracy: 0.7153 - val_loss: 0.6212 - val_accuracy: 0.7148
Epoch 5/10
64/64 [==============================] - 26s 402ms/step - loss: 0.5814 - accuracy: 0.7148 - val_loss: 0.6285 - val_accuracy: 0.7148
Epoch 6/10
64/64 [==============================] - 27s 427ms/step - loss: 0.5701 - accuracy: 0.7212 - val_loss: 0.6382 - val_accuracy: 0.7129
Epoch 7/10
64/64 [==============================] - 26s 402ms/step - loss: 0.5661 - accuracy: 0.7246 - val_loss: 0.6493 - val_accuracy